In [2]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pickle
import pandas as pd
from performance_rul import rmse, mape, re, prediction_horizon, alpha_coverage


battery_list = ['CS2_33', 'CS2_34', 'CS2_35', 'CS2_36', 'CS2_37', 'CS2_38']
cols = ['Dataset', 'RMSE', 'MAPE', 'PH', 'RE_mean', 'RE_std']
df_result_pf = pd.DataFrame(columns=cols)

for name in battery_list:        
    file_name = 'result_' + name + '.pickle'
    with open(file_name, 'rb') as f:
        [t, y, threshold, idx_ttf, idx_pred, true_ttf, max_RUL, 
            xh, yh, y_bands, rul_mean, rul_bands, rul, rul_weights, particles, w
        ] = pickle.load(f)
    
    rul_true = (true_ttf-t[idx_pred])*(true_ttf-t[idx_pred]>=0)
    rul_pred = rul_mean[rul_true>0]
    rul_true = rul_true[rul_true>0]

    metric_rmse = rmse(rul_true, rul_pred)
    metric_mape = mape(rul_true, rul_pred)
    metric_re = re(rul_true, rul_pred)
    metric_ph = prediction_horizon(rul_true, rul_pred)
    metric_alpha_coverage = alpha_coverage(rul_true, rul, rul_weights, .9)

    current_result = pd.DataFrame(data={'Dataset': name, 'RMSE': metric_rmse, 'MAPE': metric_mape, 
    'PH': metric_ph, 'RE_mean': np.mean(metric_re), 'RE_std':np.std(metric_re)}, index=[0])
    df_result_pf = pd.concat([df_result_pf, current_result]).reset_index(drop=True)      

df_result_reg = pd.DataFrame(columns=cols)
for name in battery_list:
    file_name = 'result_reg_' + name + '.pickle'
    with open(file_name, 'rb') as f:
        [t, y, threshold, idx_ttf, idx_pred, true_ttf, max_RUL, 
            x_hat, yh, y_bands, rul_mean, rul_bands, rul, x_sample
        ] = pickle.load(f)

    rul_true = (true_ttf-t[idx_pred])*(true_ttf-t[idx_pred]>=0)
    rul_pred = rul_mean[rul_true>0]
    rul_true = rul_true[rul_true>0]

    metric_rmse = rmse(rul_true, rul_pred)
    metric_mape = mape(rul_true, rul_pred)
    metric_re = re(rul_true, rul_pred)
    metric_ph = prediction_horizon(rul_true, rul_pred)
    metric_alpha_coverage = alpha_coverage(rul_true, rul, rul_weights, .9)

    current_result = pd.DataFrame(data={'Dataset': name, 'RMSE': metric_rmse, 'MAPE': metric_mape, 
    'PH': metric_ph, 'RE_mean': np.mean(metric_re), 'RE_std':np.std(metric_re)}, index=[0])
    df_result_reg = pd.concat([df_result_reg, current_result]).reset_index(drop=True)

print('Particle filter:')
print(df_result_pf)
print('\nRegression:')
print(df_result_reg)

Particle filter:
  Dataset        RMSE      MAPE     PH   RE_mean    RE_std
0  CS2_33   25.786281  0.426804  300.0  0.426804  0.895939
1  CS2_34  188.578553  1.141725  300.0  1.141725  1.068721
2  CS2_35   53.945348  0.780925  300.0  0.780925  1.103169
3  CS2_36   21.502369  0.177375  300.0  0.177375  0.193282
4  CS2_37   52.069483  0.232915  200.0  0.232915  0.149095
5  CS2_38   50.708840  0.412987  141.0  0.412987  0.594381

Regression:
  Dataset        RMSE      MAPE     PH   RE_mean    RE_std
0  CS2_33  251.675434  0.910231  150.0  0.910231  0.852452
1  CS2_34  175.445885  0.599796  220.0  0.599796  0.464818
2  CS2_35  329.662249  2.107287    0.0  2.107287  1.097334
3  CS2_36  202.356673  0.796504  200.0  0.796504  0.512175
4  CS2_37  289.213123  1.187414   60.0  1.187414  0.945858
5  CS2_38  282.318391  1.492582    0.0  1.492582  0.733419
